In [ ]:
from packages.database_tools import connect
conn, cur = connect()

import pandas as pd

In [ ]:
from collections import defaultdict

def fetch_job_results(job_id=None):
    if job_id is None:
        query = """
        SELECT * FROM games
        WHERE job_id = (SELECT job_id FROM games WHERE finished_on IS NOT NULL ORDER BY finished_on DESC LIMIT 1)
        """
    else:
        query = f"""
        SELECT * FROM games
        WHERE job_id = '{job_id}'
        """

    cur.execute(query)
    results = cur.fetchall()
    df = pd.DataFrame(results, columns=[desc[0] for desc in cur.description])
    return df

def get_deck_stats(df):
    combination_stats = defaultdict(lambda: {'games': 0, 'wins': 0})
    single_stats = defaultdict(lambda: {'games': 0, 'wins': 0})

    for _, row in df.iterrows():
        # Deck 1
        d1 = row['deck1_name']
        d1_wins = row['deck1_wins']
        if d1:
            combination_stats[d1]['games'] += row['game_count']
            combination_stats[d1]['wins'] += d1_wins
            # Sub-decks
            for part in d1.split():
                single_stats[part]['games'] += row['game_count']
                single_stats[part]['wins'] += d1_wins

        # Deck 2
        d2 = row['deck2_name']
        d2_wins = row['deck2_wins']
        if d2:
            combination_stats[d2]['games'] += row['game_count']
            combination_stats[d2]['wins'] += d2_wins
            for part in d2.split():
                single_stats[part]['games'] += row['game_count']
                single_stats[part]['wins'] += d2_wins

        # Deck 3
        d3 = row['deck3_name']
        d3_wins = row['deck3_wins']
        if d3:
            combination_stats[d3]['games'] += row['game_count']
            combination_stats[d3]['wins'] += d3_wins
            for part in d3.split():
                single_stats[part]['games'] += row['game_count']
                single_stats[part]['wins'] += d3_wins

        # Deck 4
        d4 = row['deck4_name']
        d4_wins = row['deck4_wins']
        if d4:
            combination_stats[d4]['games'] += row['game_count']
            combination_stats[d4]['wins'] += d4_wins
            for part in d4.split():
                single_stats[part]['games'] += row['game_count']
                single_stats[part]['wins'] += d4_wins

    # Convert to DataFrame
    combination_stats_df = pd.DataFrame([
        {'deck': k, 'games': v['games'], 'wins': v['wins'], 'win_rate': v['wins'] / v['games'] if v['games'] > 0 else 0}
        for k, v in combination_stats.items()
    ])
    single_stats_df = pd.DataFrame([
        {'deck': k, 'games': v['games'], 'wins': v['wins'], 'win_rate': v['wins'] / v['games'] if v['games'] > 0 else 0}
        for k, v in single_stats.items()
    ])
    combination_stats_df = combination_stats_df.sort_values(by='games', ascending=False).reset_index(drop=True)
    single_stats_df = single_stats_df.sort_values(by='games', ascending=False).reset_index(drop=True)
    return combination_stats_df, single_stats_df

def get_subdeck_stats(df):
    stats = defaultdict(lambda: {'games': 0, 'wins': 0})

    for _, row in df.iterrows():
        for i in range(1, 5):
            deck_name = row.get(f'deck{i}_name')
            deck_wins = row.get(f'deck{i}_wins')
            game_count = row.get('game_count')
            if deck_name and isinstance(deck_name, str):
                for part in deck_name.split():
                    stats[part]['games'] += game_count
                    stats[part]['wins'] += deck_wins

    stats_df = pd.DataFrame([
        {'subdeck': k, 'games': v['games'], 'wins': v['wins'], 'win_rate': v['wins'] / v['games'] if v['games'] > 0 else 0}
        for k, v in stats.items()
    ])
    stats_df = stats_df.sort_values(by='games', ascending=False).reset_index(drop=True)
    return stats_df

In [ ]:
cur.execute("SELECT job_id FROM games WHERE finished_on IS NOT NULL ORDER BY finished_on;")
job_id = cur.fetchone()[0]
job_id

In [ ]:
df = fetch_job_results(job_id)

In [ ]:
combo_df, single_df = get_deck_stats(df)

In [ ]:
combo_df.head()

In [ ]:
single_df.head()

In [ ]:
df_high_wins = df[(df['deck1_wins'] > 9) | (df['deck2_wins'] > 9)]
df_high_wins

In [13]:
decks = [
    "650dcb28-86e0-409e-9a40-d0ad16661d55",
    "abe8ffed-c4ec-4277-8890-a31e936928d2",
    "75de1fd1-092e-436e-b10f-efd5851ab8c7",
    "268c6f03-05a2-4d46-93e1-b53dab340bb0",
    "29f2f98a-d3c5-416f-b7ba-363788f4a1a7",
    "78226799-df3f-42a3-9361-568b02cfbcae",
    "c463e3cd-adea-4706-b008-b67ab0f51f09",
    "beb944dd-dfb4-460a-af1c-52523303d0db",
    "16452f0d-b1fc-4b50-bf17-5c74a67c558b",
    "13b4d508-38a0-44de-87cf-8be8b088692a",
    "3a3bd7a1-9b84-441b-9ed1-87570982ee29",
    "d810c36d-9785-4fa4-9231-ebf9b835de3f",
    "6f42b0ca-74f8-4cef-87be-0b2ffc02bcd1",
    "aa7b4971-db0b-40fe-86fe-fdaf9f245852",
    "1d223394-5d76-4e8c-bcca-26aaed8ff17a",
    "9671e667-d51c-49a1-9db9-ab8ee9477076",
    "dfbda714-94e1-4915-9e62-b12d1958ce38",
    "49cdcc2c-0249-44c5-a802-ed595a971e0e",
    "a3da8d2a-e2dd-4aae-92cd-7bc0a738d3e6",
    "f38c3304-9c82-4c85-a9d7-153e96871a4c",
]


In [14]:
import random
from collections import defaultdict
from itertools import combinations

def balanced_partners(deck_ids, k):
    partners = defaultdict(list)
    partner_count = defaultdict(int)

    for d in deck_ids:
        choices = sorted(
            [x for x in deck_ids if x != d],
            key=lambda x: partner_count[x]
        )

        selected = choices[:k]
        partners[d] = selected

        for p in selected:
            partner_count[p] += 1

    return dict(partners)

def build_full_decks(partners):
    seen = set()
    full_decks = []

    for a, ps in partners.items():
        for b in ps:
            deck = tuple(sorted((a, b)))
            if deck not in seen:
                seen.add(deck)
                full_decks.append(deck)

    return full_decks

def shares_half(deck1, deck2):
    return bool(set(deck1) & set(deck2))

def sample_matches_no_mirror(full_decks, m):
    all_pairs = list(combinations(full_decks, 2))
    random.shuffle(all_pairs)

    match_count = {d: 0 for d in full_decks}
    matches = []

    for d1, d2 in all_pairs:
        # ❌ skip mirror-half matches
        if shares_half(d1, d2):
            continue

        if match_count[d1] < m and match_count[d2] < m:
            matches.append((d1, d2))
            match_count[d1] += 1
            match_count[d2] += 1

        if all(c >= m for c in match_count.values()):
            break

    return matches

In [15]:
pairings = balanced_partners(decks, 4)
full_decks = build_full_decks(pairings)
matches = sample_matches_no_mirror(full_decks, 6)

In [ ]:
import uuid
import datetime

job_id = uuid.uuid4()
game_count = 1
created_on = datetime.datetime.now(datetime.timezone.utc)
format = 'jumpstart'

values_sql = []

for match in matches:
    (a, b), (c, d) = match
    primary_key = uuid.uuid4()

    values_sql.append(
        f"('{primary_key}', '{a}', '{b}', '{c}', '{d}', "
        f"'{job_id}', {game_count}, '{format}', '{created_on.isoformat()}')"
    )

sql_statement = f"""
    INSERT INTO "public"."games" (
        primary_key,
        deck_version_id1,
        deck_version_id2,
        deck_version_id3,
        deck_version_id4,
        job_id,
        game_count,
        format,
        created_on
    )
    VALUES
    {",\n    ".join(values_sql)};
"""


In [23]:
print(sql_statement)


    INSERT INTO "public"."games" (
        primary_key, 
        deck_version_id1,
        deck_version_id2,
        deck_version_id3,
        deck_version_id4,
        job_id,
        game_count,
        format,
        created_on
    )
    VALUES
    ('e2ca49c7-bb9b-4555-9ca0-49bba508d60e', '16452f0d-b1fc-4b50-bf17-5c74a67c558b', '75de1fd1-092e-436e-b10f-efd5851ab8c7', '3a3bd7a1-9b84-441b-9ed1-87570982ee29', 'abe8ffed-c4ec-4277-8890-a31e936928d2', 'e01a9791-a042-4257-a570-1a24965305d9', 1, 'jumpstart', '2025-12-21T21:44:02.385459+00:00'),
    ('7627f3d1-986d-4d6e-bc18-8e98fbccb9eb', 'aa7b4971-db0b-40fe-86fe-fdaf9f245852', 'dfbda714-94e1-4915-9e62-b12d1958ce38', '49cdcc2c-0249-44c5-a802-ed595a971e0e', 'd810c36d-9785-4fa4-9231-ebf9b835de3f', 'e01a9791-a042-4257-a570-1a24965305d9', 1, 'jumpstart', '2025-12-21T21:44:02.385459+00:00'),
    ('9b7ca3f6-9067-4f77-83b2-03cdde0c181b', '78226799-df3f-42a3-9361-568b02cfbcae', 'd810c36d-9785-4fa4-9231-ebf9b835de3f', 'a3da8d2a-e2dd-4aae-92cd-7bc0